In [ ]:
import math
import os
import regex as re
from random import sample
from textwrap import dedent
from typing import Literal, Union

from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import AIMessage
from langchain_openai.chat_models import ChatOpenAI

In [ ]:

load_dotenv()

In [ ]:
alphabet = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz!@#$%^&*()-_=+[]{}|;'

def create_sudoku(base: int = 3) -> np.ndarray:
    def pattern(r,c): return (base * (r % base) + r // base + c) % (base*base)

    # randomize rows, columns and numbers (of valid base pattern)
    def shuffle(s): return sample(s,len(s)) 
    rBase = range(base) 
    rows  = [g*base + r for g in shuffle(rBase) for r in shuffle(rBase)] 
    cols  = [g*base + c for g in shuffle(rBase) for c in shuffle(rBase)]
    nums  = shuffle(range(1,base*base+1))
    
    board = [[nums[pattern(r,c)] for c in cols] for r in rows]
    return np.array(board) 

In [ ]:
def draw_sudoku(sudoku: np.ndarray) -> str:
    return '\n'.join([''.join([alphabet[x] for x in row]) for row in sudoku])

def draw_sudoku_pretty(sudoku: np.ndarray) -> str:
    size = sudoku.shape[0]
    base = int(math.sqrt(size))
    lines = []
    for i in range(size):
        if i % base == 0 and i != 0:
            lines.append('-' * (size + base - 1))
        line = ''
        for j in range(size):
            if j % base == 0 and j != 0:
                line += '|'
            line += alphabet[sudoku[i, j]]
        lines.append(line)
    return '\n'.join(lines)

def remove_numbers(sudoku: np.ndarray, level: Literal['easy', 'medium', 'hard', 'extreme']) -> np.ndarray:
    level_dict = {'easy':(0.38, 0.51), 'medium':(0.51, 0.63), 'hard':(0.63, 0.69), 'extreme':(0.69, 0.77)}
    lower, upper = level_dict[level]
    lower, upper = lower * sudoku.size, upper * sudoku.size
    to_remove = np.random.randint(lower, upper)
    indices = np.random.choice(range(sudoku.size), to_remove, replace=False)
    sudoku[indices // sudoku.shape[0], indices % sudoku.shape[0]] = 0
    return sudoku
   
def generate_sudoku(level: Literal['easy', 'medium', 'hard', 'extreme'], base: int = 3) -> np.ndarray:
    sudoku = create_sudoku(base)
    sudoku = remove_numbers(sudoku, level)
    return sudoku

In [ ]:
def validate_structure(skeleton: np.ndarray, sudoku: np.ndarray) -> bool:
    indices = np.where(skeleton != 0)
    return np.all(sudoku[indices] == skeleton[indices])

def validate_solution(skeleton: np.ndarray, sudoku: np.ndarray) -> float:
    return np.sum(sudoku == skeleton) / skeleton.size

def is_valid(solution: np.ndarray, sudoku: np.ndarray) -> bool:
    return np.all(solution == sudoku)

In [ ]:
class Game:
    def __init__(self, chat_model: BaseChatModel, initial_size: int = 2):
        self.chat_model = chat_model
        self.initial_size = initial_size
        
    def play(self):
        template = dedent("""
<Instructions>
You will be solving a sudoku puzzle.

The sudoku grid is provided in a square format, where each line represents a row in the grid, and each character represents a cell in that row, with no spaces between characters.

The size of the sudoku grid is {grid_size}x{grid_size}.

The size of each box in the sudoku grid is {box_size}x{box_size}.

The symbols representing the numbers are: {alphabet}

Empty cells in the sudoku grid are represented by zeros.

The rules of sudoku are:
1. Each row must contain each symbol exactly once
2. Each column must contain each symbol exactly once
3. Each {box_size}x{box_size} box must contain each symbol exactly once

Here is an example of a 9x9 sudoku puzzle with 3x3 boxes and its solution in the specified format:

<example>
Puzzle:
003020600
900305001
001806400
008102900
700000008
006708200
002609500
800203009
005010300

Solution:
483921657
967345821
251876493
548132976
729564138
136798245
372689514
814253769
695417382
</example>

Now, solve the provided sudoku puzzle by following these steps:

1. First, fill in any cells where there is only one possible symbol that can go there based on the existing symbols in that cell's row, column and box.

2. If no cells can be definitively filled in, use pencil marks to keep track of candidate symbols for each blank cell (cells with zeros) by eliminating symbols already present in that cell's row, column and box.

3. Look for patterns like single candidates, naked pairs/triples, hidden pairs/triples, X-Wings, etc. to logically eliminate candidates and fill in cells.

4. Repeat steps 1-3, logically filling in symbols one at a time, until the puzzle is completely solved and there are no more zeros remaining.

5. Double check that your solution follows all 3 sudoku rules listed above to confirm the puzzle has been properly solved.

6. Output the final solved grid in the same square format as the input, with each line representing a row in the grid, and each character representing a cell in that row, with no spaces between characters. Enclose the output inside <solved_sudoku> tags.

Remember, only use the symbols given in {alphabet}. The solution mustn't contain any zeros. The solution must 

Solve the sudoku puzzle now.

<sudoku>
{sudoku}
</sudoku>

</Instructions>""")
        
        prompt = PromptTemplate.from_template(template)
        
        def parse(message: AIMessage) -> Union[np.ndarray, None]:
            text = message.content
            sudoku_text = re.search(r'(?<=<solved_sudoku>\n)(.*?)(?=\n</solved_sudoku>)', text, re.DOTALL).group(1)  
            #check if sudoku_text contains only valid characters from variable alphabet or \n
            if re.search(r'[^' + alphabet + '\n]', sudoku_text):
                return None
            sudoku = np.array([[alphabet.index(x)for x in row] for row in sudoku_text.split('\n')])
            return sudoku
        
        chain = prompt | self.chat_model | parse

        for i in range(self.initial_size, 10):
            size = i * i
            current_alphabet = alphabet[1:size + 1]
            should_continue = True
            for level in ['easy', 'medium', 'hard', 'extreme']:
                solution = create_sudoku(i)
                skeleton = remove_numbers(solution.copy(), level)
                sudoku_answer = chain.invoke({
                    'box_size': i,
                    'grid_size': i * i,
                    'alphabet': current_alphabet,
                    'sudoku': draw_sudoku(skeleton)
                })
                if sudoku_answer is None:
                    # total failure
                    print(f'Level: {level}, Size: {i}, Failure')
                
                if is_valid(solution, sudoku_answer):
                    print(f'Level: {level}, Size: {i}, Success')
                else:
                    
                    structure = validate_structure(skeleton, sudoku_answer)
                    percentage = validate_solution(solution, sudoku_answer)
                    
                    print(f'Level: {level}, Size: {i}, Structure: {structure}, Percentage: {percentage}')
                    print(f'Skeleton:\n{draw_sudoku_pretty(skeleton)}\n')
                    print(f'Solution:\n{draw_sudoku_pretty(solution)}\n')
                    print(f'Result:\n{draw_sudoku_pretty(sudoku_answer)}')
                    should_continue = False
                    break
            if not should_continue:
                break

In [ ]:
chat_model = ChatAnthropic(api_key=os.environ['ANTHROPIC_KEY'], model_name='claude-3-sonnet-20240307')

game = Game(chat_model, 3)
game.play()